Creating data frame for all data sets

In [2]:
import pandas as pd

dim_city = pd.read_csv('E:/Projects/Pandas/Provide Insights to Chief of Operations in Transportation Domain/RPC13_Input_For_Participants/datasets/csv_files/dim_city.csv')

dim_date = pd.read_csv('E:/Projects/Pandas/Provide Insights to Chief of Operations in Transportation Domain/RPC13_Input_For_Participants/datasets/csv_files/dim_date.csv')

city_target_passenger_rating = pd.read_csv('E:/Projects/Pandas/Provide Insights to Chief of Operations in Transportation Domain/RPC13_Input_For_Participants/datasets/csv_files/city_target_passenger_rating.csv')

dim_repeat_trip_distribution = pd.read_csv('E:/Projects/Pandas/Provide Insights to Chief of Operations in Transportation Domain/RPC13_Input_For_Participants/datasets/csv_files/dim_repeat_trip_distribution.csv')

fact_passenger_summary = pd.read_csv('E:/Projects/Pandas/Provide Insights to Chief of Operations in Transportation Domain/RPC13_Input_For_Participants/datasets/csv_files/fact_passenger_summary.csv')

monthly_target_new_passengers = pd.read_csv('E:/Projects/Pandas/Provide Insights to Chief of Operations in Transportation Domain/RPC13_Input_For_Participants/datasets/csv_files/monthly_target_new_passengers.csv')

monthly_target_trips = pd.read_csv('E:/Projects/Pandas/Provide Insights to Chief of Operations in Transportation Domain/RPC13_Input_For_Participants/datasets/csv_files/monthly_target_trips.csv')

fact_trips = pd.read_csv('E:/Projects/Pandas/Provide Insights to Chief of Operations in Transportation Domain/RPC13_Input_For_Participants/datasets/csv_files/fact_trips.csv')


In [ ]:
print(dim_city.head())
# print(dim_date.head())
# print(city_target_passenger_rating.head())
# print(dim_repeat_trip_distribution.head())
# print(fact_passenger_summary.head())
# print(monthly_target_new_passengers.head())
# print(monthly_target_trips.head())


1. Top and Bottom Performing Cities
• Identify the top 3 and bottom 3 cities by total trips over the entire analysis
period.

In [ ]:
cities_trips_data = pd.merge(dim_city, fact_trips, how='inner', on='city_id')

monthly_data.keys()

top_3_cities = pd.DataFrame(monthly_data.groupby('city_name')['trip_id'].count().sort_values(ascending=False).head(3))
bottom_3_cities = pd.DataFrame(monthly_data.groupby('city_name')['trip_id'].count().sort_values(ascending=True).head(3))

print(top_3_cities, bottom_3_cities)


2. Average Fare per Trip by City
• Calculate the average fare per trip for each city and compare it with the city's
average trip distance. Identify the cities with the highest and lowest average
fare per trip to assess pricing efficiency across locations.

In [ ]:
cities_trips_data = pd.merge(dim_city, fact_trips, how='inner', on='city_id')

cities_trips_data['avg_fare'] = cities_trips_data.groupby('city_name')['fare_amount'].transform('mean').sort_values(ascending=False).round(2)

cities_trips_data['avg_distance'] = cities_trips_data.groupby('city_name')['distance_travelled(km)'].transform('mean').sort_values(ascending=True).round(2)

cities_trips_data['avg_fare_per_km'] = (cities_trips_data['avg_fare']/cities_trips_data['avg_distance']).round(2)

cities_trips_data[['city_name', 'avg_fare', 'avg_distance', 'avg_fare_per_km']].drop_duplicates().reset_index(drop=True)




3. Average Ratings by City and Passenger Type
• Calculate the average passenger and driver ratings for each city, segmented
by passenger type (new vs. repeat). Identify cities with the highest and lowest
average ratings.

In [ ]:
cities_rating  = pd.merge(dim_city, fact_trips, how='inner', on= 'city_id')

cities_rating['avg_passenger_rating'] = cities_rating.groupby(['city_id','passenger_type'])['passenger_rating'].transform('mean').round(2)

cities_rating['avg_driver_rating'] = cities_rating.groupby(['city_id','passenger_type'])['driver_rating'].transform('mean').round(2)

cities_rating[['city_name','avg_passenger_rating','avg_driver_rating','passenger_type']].drop_duplicates().reset_index(drop=True).sort_values(by=['avg_passenger_rating','avg_driver_rating'], ascending=True)

4. Peak and Low Demand Months by City
e For each city, identify the month with the highest total trips (peak demand)
and the month with the lowest total trips (low demand). This analysis will help
Goodcabs understand seasonal patterns and adjust resources accordingly.

In [ ]:
monthly_data = pd.merge(dim_city, fact_trips, how='inner', on='city_id')

# monthly_data.keys()

demand_details = pd.merge(monthly_data, dim_date, how='inner', on='date')

# demand_details.keys()

demand = pd.DataFrame(demand_details.groupby(['city_name','month_name'])['trip_id'].count())

demand.reset_index(drop=True)

max_demand = demand.loc[demand.groupby('city_name')['trip_id'].idxmax()]

low_demand = demand.loc[demand.groupby('city_name')['trip_id'].idxmin()]

max_demand
low_demand




5. Weekend vs. Weekday Trip Demand by City
• Compare the total trips taken on weekdays versus weekends for each city
over the six-month period. Identify cities with a strong preference for either
weekend or weekday trips to understand demand variations.

In [ ]:
weekday_details = pd.DataFrame(demand_details[demand_details['day_type'] == 'Weekday'].groupby('city_name')['trip_id'].count())
weekday_details.rename(columns={'trip_id': 'count(trip_id)'}, inplace=True)
weekend_details = pd.DataFrame(demand_details[demand_details['day_type'] == 'Weekend'].groupby('city_name')['trip_id'].count())
weekend_details.rename(columns={'trip_id': 'count(trip_id)'}, inplace=True)

display(weekday_details, weekend_details)

6. Repeat Passenger Frequency and City Contribution Analysis
• Analyse the frequency of trips taken by repeat passengers in each city
(e.g., % of repeat passengers taking 2 trips, 3 trips, etc.). Identify which cities
contribute most to higher trip frequencies among repeat passengers, and
examine if there are distingu ishable pattems between tourism-focused and
business-focused cities.

In [ ]:
repeat_passenger_details = pd.merge(dim_city, dim_repeat_trip_distribution, how='inner', on='city_id')

repeat_passenger_details['RPC_based_on_cities'] = repeat_passenger_details.groupby(['city_name','trip_count'])['repeat_passenger_count'].transform('sum')

repeat_passenger_details['RPC_based_on_trip_count'] = repeat_passenger_details.groupby('trip_count')['repeat_passenger_count'].transform('sum')

repeat_passenger_details['frequency_%'] = ((repeat_passenger_details['RPC_based_on_cities']/repeat_passenger_details['RPC_based_on_trip_count'])*100).round(2)

repeat_passenger_details[['city_name', 'trip_count','frequency_%']].drop_duplicates().reset_index(drop=True)










7. Monthly Target Achievement Analysis for Key Metrics
• For each city, evaluate monthly performance against targets for total trips,
new passengers, and average passenger ratings from targets db. Determine
if each metric met, exceeded, or missed the target, and calcu late the
percentage difference. Identify any consistent pattems in target achievement,
particularly across tourism versus business-focused cities.

In [ ]:
# Merge city and trips data
result1 = pd.merge(dim_city, fact_trips, how='inner', on='city_id')

# Merge date and trips data
result2 = pd.merge(dim_date, fact_trips, how='inner', on='date')

# Merge result1 and result2 based on trip_id
result3 = pd.merge(result1, result2, how='inner', on='trip_id')

result4 = result3[['city_id_y', 'city_name', 'trip_id', 
                   'date_y', 'start_of_month', 'month_name', 'day_type',
                   'passenger_type_y', 'distance_travelled(km)_y',
                   'fare_amount_y', 'passenger_rating_y', 'driver_rating_y']]

# Create a dictionary to rename the columns by removing 'y' from the column names
rename_dict = {
    'city_id_y': 'city_id',
    'date_y': 'date',
    'passenger_type_y': 'passenger_type',
    'distance_travelled(km)_y': 'distance_travelled(km)',
    'fare_amount_y': 'fare_amount',
    'passenger_rating_y': 'passenger_rating',
    'driver_rating_y': 'driver_rating'
}

# Rename the columns using the dictionary
result4.rename(columns=rename_dict, inplace=True)

result4['total_trip_count_based_on_cities'] = result4.groupby(['city_id','city_name','month_name'])['trip_id'].transform('count')
# pd.DataFrame(result5.head(3))

In [ ]:

Trip_performance_analysis = result4[['city_id', 'city_name','start_of_month', 'month_name','total_trip_count_based_on_cities']].drop_duplicates().reset_index(drop=True)

Trip_performance_analysis = pd.merge(Trip_performance_analysis,monthly_target_trips,how='inner',left_on=['city_id','start_of_month'], right_on=['city_id','month'])

Trip_performance_analysis['result'] =    Trip_performance_analysis['total_trip_count_based_on_cities'] - Trip_performance_analysis['total_target_trips']


Trip_performance_analysis['percentage_difference'] =   (((Trip_performance_analysis['total_trip_count_based_on_cities'] - Trip_performance_analysis['total_target_trips'])/Trip_performance_analysis['total_target_trips'])*100).round(2)

# Trip_performance_analysis[Trip_performance_analysis['city_name']=='Jaipur'][['city_name','total_target_trips','result']]

# Trip_performance_analysis.groupby('city_name')['result'].sum()
Trip_performance_analysis

In [ ]:
passenger_analysis = pd.merge(dim_city,fact_passenger_summary,how='inner',on='city_id')

passenger_analysis = pd.merge(passenger_analysis,monthly_target_new_passengers, how='inner',on=['month','city_id'])

passenger_analysis.head()

# passenger_analysis.drop(columns='city_id_x')

passenger_analysis = pd.DataFrame(passenger_analysis.groupby('city_name')[['new_passengers','target_new_passengers']].sum())

passenger_analysis['result'] = passenger_analysis['new_passengers'] - passenger_analysis['target_new_passengers']

passenger_analysis ['percentage_difference']= (((passenger_analysis['new_passengers'] - passenger_analysis['target_new_passengers'] )/ passenger_analysis['target_new_passengers']) * 100).round(2)

passenger_analysis

In [ ]:
rating_analysis = pd.merge(fact_trips,city_target_passenger_rating,how='inner',on='city_id')

rating_analysis = pd.DataFrame(rating_analysis.groupby('city_id')[['passenger_rating','target_avg_passenger_rating']].mean().round(2))
rating_analysis ['percentage_target'] = (((rating_analysis['passenger_rating'] - rating_analysis['target_avg_passenger_rating'])/rating_analysis['target_avg_passenger_rating'])*100).round(2)

rating_analysis = pd.merge(rating_analysis,dim_city,how='inner',on='city_id')

rating_analysis

8. Highest and Lowest Repeat Passenger Rate (RPR%) by City and Month    
Analyse the Repeat Passenger Rate (RPR%) for each city across the six-
month period. Identify the top 2 and bottom 2 cities based on their RPR% to
determine which locations have the strongest and weakest rates.
Similarly, analyse the RPR% by month across all cities and identify the
months with the highest and lowest repeat passenger rates. This will help to
pinpoint any seasonal patterns or months with higher repeat passenger
loyalty.

In [ ]:
RPR_details = pd.merge(dim_city,fact_passenger_summary,how='inner',on='city_id')

RPR_details = RPR_details[['city_id','city_name','repeat_passengers','total_passengers']]

RPR_details = RPR_details.groupby('city_id')[['repeat_passengers','total_passengers']].sum()

RPR_details['result'] = (((RPR_details['repeat_passengers']/RPR_details['total_passengers'])*100).round(2))

RPR_details.head(5)

• Similarly, analyse the RPR% by month across all cities and identify the
months with the highest and lowest repeat passenger rates. This will help to
pinpoint any seasonal patterns or months with higher repeat passenger
loyalty.

In [ ]:
RPR_month_details = pd.merge(dim_city,fact_passenger_summary,how='inner',on='city_id')

RPR_month_details = RPR_month_details[['city_id','city_name','month','repeat_passengers','total_passengers']]

RPR_month_details = RPR_month_details.groupby(['city_id','month'])[['repeat_passengers','total_passengers']].sum()

RPR_month_details['result'] = (((RPR_month_details['repeat_passengers']/RPR_month_details['total_passengers'])*100).round(2))

RPR_month_details